# Following hf book lets see about generating a movie summary



first need to get the data out. most of the scripts are stored as .tar.gz but don't want to extract all yet. 

In a script, "EXT." is an abbreviation for "exterior." It is a scene heading or slugline that indicates the location of the scene taking place outside, typically outdoors or in an exterior setting. Scene headings are used in screenplay formatting to describe the location and time of each scene.

In a script, "INT." is an abbreviation for "interior." It is a scene heading or slugline that indicates the location of the scene taking place indoors or in an interior setting.

A useful model may be combining sumarrys from multiple reviews.

Character cues (e.g., MAN, CATHERINE) preface the lines the actors
speak. Action/description lines describe what the camera sees (e.g., We can’t get a
good glimpse of his face, but his body: : : ).

In [9]:
data_directory = '/home/fergus/data/scriptbase/scriptbase_j'

script_filename = f'{data_directory}/sw3/processed/script_clean.txt'
with open(script_filename) as file:
    script = file.read()
    
#script


# Preprocessing the text. 

For now the scripts have a consistent structure, which will mean making a consistent preprocessing method easy. 
I'm going to begin by removing all scene descriptions, and just deal with the dialogue of the text. This will be a key difficult of any future system, this will be limited to the information found in the dialogue. 

In [41]:

processed_script = []

# First process to character and speech
lines = []
for line in script.split('\n\n'):
    line = line.replace('     ','-')
    if line[:2] == '--':
        lines.append(line)

# print the first few lines 
for i, line in enumerate(lines):
    if i > 2:
        break
    print(line)
    print('\n\n')
    
for t, line in enumerate(lines):
    split = line.split('\n')
    character = split[0].replace('-','').lower()
    speech = ""
    for text in split[1:]:
        text = text.replace('-','').replace('. . .', '').lower() + ' '
        speech += text
    d = {'character':character, 'text':speech}
    processed_script.append(d)

print(processed_script[:10])

----ANAKIN
--There isn't a droid made that can out fly you, Master, and no other way to get to the Chancellor . . .



----OBI-WAN
--Look out, four droids inbound . . .



----OBI-WAN
--We've got to split them up.



[{'character': 'anakin', 'text': "there isn't a droid made that can out fly you, master, and no other way to get to the chancellor  "}, {'character': 'obiwan', 'text': 'look out, four droids inbound  '}, {'character': 'obiwan', 'text': "we've got to split them up. "}, {'character': 'anakin', 'text': 'break left, fly through the guns on that tower. '}, {'character': 'obiwan', 'text': 'easy for you to say  why am i always the bait? '}, {'character': 'anakin', 'text': "don't worry. i'm coming around behind you. "}, {'character': 'obiwan', 'text': "anakin, they're all over me! "}, {'character': 'anakin', 'text': 'dead ahead! closing  lock onto him, artoo  '}, {'character': 'anakin', 'text': '(continuing, laughs) we got him, artoo! '}, {'character': 'anakin', 'text': 'i copy, a

### There's some very basic preprocessing now. I wonderhow many tokens there are


In [58]:
from transformers import pipeline, set_seed
pipe = pipeline("summarization", model="t5-large")



/home/fergus/.local/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [59]:
processed_script_as_text = ''
for p in processed_script:
    processed_script_as_text += p['text']
print(f'number of characters = {len(processed_script_as_text)}') # numer of characters
print(f'number of words = {len(processed_script_as_text.split())}') # numer of characters

number of characters = 68190
number of words = 12524


#### Tokenize the script

In [65]:
# from nltk.tokenize import sent_tokenize
# tokenized_script = sent_tokenize(processed_script_as_text)
# tokenized_script[:10]

tokenized_script = ' '.join(processed_script_as_text.split()[:256])
tokenized_script

"there isn't a droid made that can out fly you, master, and no other way to get to the chancellor look out, four droids inbound we've got to split them up. break left, fly through the guns on that tower. easy for you to say why am i always the bait? don't worry. i'm coming around behind you. anakin, they're all over me! dead ahead! closing lock onto him, artoo (continuing, laughs) we got him, artoo! i copy, artoo. i'm going down on the deck. good idea ... i need some room to maneuver. (continuing) cut right. do you hear me?! cut right. don't let him get a handle on you. come on, artoo, lock on! lock on! hurry up! i don't like this! (continuing) ouch! (continuing) don't even try to fix it, arfour. i've shut it down. we're locked on ... we've got him (continuing) yeah! we got him good going, artoo. next time you're the bait now let's find the command ship and get on with it ... lock onto them, artoo. master, general grievous's ship is directly ahead. (continuing) the one crawling with vu

In [66]:
pipe_out = pipe(tokenized_script)

In [67]:
pipe_out[0]["summary_text"].replace(" .<n>",".\n")

"general grievous's ship is directly ahead, crawling with vulture droids . sfoils in attack position, mark your position and form your squad up behind me . anakin, they're all over me! dead ahead! i copy, artoo, i'm going down on the deck ."

# Testing scene extraction

Scene extraction is possible based off the raw te4xt. It will have very different preprocessing because the previous chunk of worked looked only at dialogue. 

EXT. and INT. Mark change of scenes 

In [62]:
def scene_to_dialogue_and_setup(scene):
    lines = []
    setup = []
    for line in scene.split('\n\n'):
        line = line.replace('     ','-')
        if line[:2] == '--':
            lines.append(line)
        else:
            setup.append(line)
    

    processed_scene= []
    for t, line in enumerate(lines):
        split = line.split('\n')
        character = split[0].replace('-','').lower()
        speech = ""
        for text in split[1:]:
            text = text.replace('-','').replace('. . .', '').lower() + ' '
            speech += text
        d = {'character':character, 'text':speech}
        processed_scene.append(d)
    return processed_scene, '\n'.join(setup)

def scene_to_setup(scene):
    lines = []
    for line in scene.split('\n\n'):
        if '\n' in line:
            break
        lines.append(line)
    return ' '.join(lines)
        

scene 5 has some good examples of identifies (like EXT or INT) not being in the script, but some non-dialogue e.g. narration or scene-setting takes place. 

In [63]:


split_scenes = [scene.split('INT.') for scene in script.split('EXT.')]
split_scenes = [inner for outer in split_scenes for inner in outer]
for i,scene in enumerate(split_scenes):
    # d = scene_to_dialogue(scene)
    if i<5:
        continue
    print('*********************************************')
    print(i)
    print(f'scene={scene}')
    dialogue, setup = scene_to_dialogue_and_setup(scene)
    print(f'setup={setup}')
    print(f'dialogue={dialogue}')
    
    if i >= 5:
        break
    
        
    
    
    

*********************************************
5
scene= OBI-WAN'S STARFIGHTER COCKPIT-SPACE

OBI-WAN struggles to maintain control of his ship.

                    OBI-WAN
          We've got to split them up.

                    ANAKIN
          Break left, fly through the guns on that tower.

OBI-WAN flies to the left of a huge tower on a REPUBLIC CRUISER. The TWO DROID DROP FIGHTERS follow.

                    OBI-WAN
          Easy for you to say . . . why am I always the bait?

                    ANAKIN
          Don't worry. I'm coming around behind you.

OBI-WAN deftly maneuvers around a large Starship's superstructure, but the TWO DROID FIGHTERS stay on his tail, BLASTING him with intense laser fire.

                    OBI-WAN
          Anakin, they're all over me!

                    ANAKIN
          Dead ahead! Closing . . . lock onto him, Artoo . . .

ARTOO BEEPS his reply as ANAKIN swoops in for the kill. ANAKIN BLASTS one of the DROID DROP FIGHTERS. It EXPLODES.

   

In [64]:
setup

" OBI-WAN'S STARFIGHTER COCKPIT-SPACE\nOBI-WAN struggles to maintain control of his ship.\nOBI-WAN flies to the left of a huge tower on a REPUBLIC CRUISER. The TWO DROID DROP FIGHTERS follow.\nOBI-WAN deftly maneuvers around a large Starship's superstructure, but the TWO DROID FIGHTERS stay on his tail, BLASTING him with intense laser fire.\nARTOO BEEPS his reply as ANAKIN swoops in for the kill. ANAKIN BLASTS one of the DROID DROP FIGHTERS. It EXPLODES.\nANAKIN BLASTS away at the second DROID DROP FIGHTER as ARTOO BEEPS an angry warning.\nOBI-WAN dives toward the surface of one of the larger TRADE FEDERATION BATTLESHIPS and is forced to fly through a maelstrom of laser flack. He skims the surface, followed by the DROID DROP FIGHTER, which is followed by ANAKIN.\nARTOO BEEPS. The crosshairs merge on the DROID DROP FIGHTER.\nOBI-WAN flies through a narrow gap between two towers on a Battleship. The DROID DROP FIGHTER hits one of Obi-Wan's wings with a laser blast, and parts of the ship 

In [69]:
from transformers import pipeline, set_seed
pipe = pipeline("summarization", model="gpt2-xl")

pipe_out = pipe(setup)

The model 'GPT2LMHeadModel' is not supported for summarization. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 398, but `max_length` is set to 20. This can lead to u

In [70]:
pipe_out

[{'summary_text': " OBI-WAN'S STARFIGHTER COCKPIT-SPACE\nOBI-WAN struggles to maintain control of his ship.\nOBI-WAN flies to the left of a huge tower on a REPUBLIC CRUISER. The TWO DROID DROP FIGHTERS follow.\nOBI-WAN deftly maneuvers around a large Starship's superstructure, but the TWO DROID FIGHTERS stay on his tail, BLASTING him with intense laser fire.\nARTOO BEEPS his reply as ANAKIN swoops in for the kill. ANAKIN BLASTS one of the DROID DROP FIGHTERS. It EXPLODES.\nANAKIN BLASTS away at the second DROID DROP FIGHTER as ARTOO BEEPS an angry warning.\nOBI-WAN dives toward the surface of one of the larger TRADE FEDERATION BATTLESHIPS and is forced to fly through a maelstrom of laser flack. He skims the surface, followed by the DROID DROP FIGHTER, which is followed by ANAKIN.\nARTOO BEEPS. The crosshairs merge on the DROID DROP FIGHTER.\nOBI-WAN flies through a narrow gap between two towers on a Battleship. The DROID DROP FIGHTER hits one of Obi-Wan's wings with a laser blast, and 